In [1]:
import numpy as np
import pandas as pd

In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [132]:
dataset = pd.read_csv("C:\\Users\\H P\\Desktop\\data\\10_udemy vahit kurs şam\\NLP\\metin_madenciligi\\train.tsv", sep = "\t")

In [133]:
dataset.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [134]:
dataset["Sentiment"].replace(0, value = "negatif", inplace = True)
dataset["Sentiment"].replace(1, value = "negatif", inplace = True)
dataset["Sentiment"].replace(3, value = "pozitif", inplace = True)
dataset["Sentiment"].replace(4, value = "pozitif", inplace = True)

In [135]:
dataset.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [136]:
dataset = dataset[(dataset.Sentiment == "negatif") | (dataset.Sentiment == "pozitif")]

In [137]:
dataset.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
21,22,1,good for the goose,pozitif
22,23,1,good,pozitif
33,34,1,"the gander , some of which occasionally amuses...",negatif
46,47,1,amuses,pozitif


In [138]:
dataset["Sentiment"].replace("negatif", value = 0, inplace = True)
dataset["Sentiment"].replace("pozitif", value = 1, inplace = True)

In [139]:
dataset.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,0
21,22,1,good for the goose,1
22,23,1,good,1
33,34,1,"the gander , some of which occasionally amuses...",0
46,47,1,amuses,1


In [140]:
target = dataset['Sentiment'].values.tolist()
data = dataset['Phrase'].values.tolist()

In [141]:
target[:10]

[0, 1, 1, 0, 1, 0, 0, 1, 1, 1]

In [142]:
data[:10]

['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
 'good for the goose',
 'good',
 'the gander , some of which occasionally amuses but none of which amounts to much of a story',
 'amuses',
 'but none of which amounts to much of a story',
 'none of which amounts to much of a story',
 'This quiet , introspective and entertaining independent is worth seeking .',
 'This quiet , introspective and entertaining independent',
 'quiet , introspective and entertaining independent']

In [143]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [144]:
x_train[100]

"'s little to recommend Snow Dogs , unless one considers cliched dialogue and perverse escapism a source of high hilarity"

In [145]:
x_train[800]

"'s a pretty good execution of a story that 's a lot richer than the ones Hollywood action screenwriters usually come up with on their own ."

In [146]:
y_train[800]

1

In [147]:
set(y_train)

{0, 1}

In [148]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [149]:
tokenizer.fit_on_texts(data)

In [101]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'to': 5,
 "'s": 6,
 'is': 7,
 'that': 8,
 'in': 9,
 'it': 10,
 'as': 11,
 'with': 12,
 'for': 13,
 'an': 14,
 'film': 15,
 'its': 16,
 'movie': 17,
 'this': 18,
 'but': 19,
 'be': 20,
 'you': 21,
 'on': 22,
 "n't": 23,
 'by': 24,
 'more': 25,
 'one': 26,
 'not': 27,
 'at': 28,
 'than': 29,
 'from': 30,
 'has': 31,
 'about': 32,
 'his': 33,
 'all': 34,
 'are': 35,
 'so': 36,
 'like': 37,
 'or': 38,
 'have': 39,
 'good': 40,
 'out': 41,
 'too': 42,
 'most': 43,
 'story': 44,
 "'": 45,
 'up': 46,
 'into': 47,
 'who': 48,
 'rrb': 49,
 'funny': 50,
 'characters': 51,
 'no': 52,
 'i': 53,
 'comedy': 54,
 'lrb': 55,
 'well': 56,
 'if': 57,
 'just': 58,
 'much': 59,
 'time': 60,
 'does': 61,
 'can': 62,
 'what': 63,
 'even': 64,
 'some': 65,
 'your': 66,
 'very': 67,
 'will': 68,
 'their': 69,
 'little': 70,
 'bad': 71,
 'life': 72,
 'way': 73,
 "''": 74,
 'which': 75,
 'been': 76,
 'best': 77,
 'make': 78,
 'any': 79,
 'love': 80,
 'enough': 81,
 'the

In [150]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [103]:
x_train[100]

"'s little to recommend Snow Dogs , unless one considers cliched dialogue and perverse escapism a source of high hilarity"

In [104]:
print(x_train_tokens[100])

[6, 70, 5, 836, 1366, 2223, 3438, 26, 5272, 1192, 170, 3, 2109, 2110, 2, 1495, 4, 230, 2960]


In [151]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [152]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [153]:
np.mean(num_tokens)

8.606775804806611

In [154]:
np.max(num_tokens)

49

In [155]:
np.argmax(num_tokens)

28184

In [156]:
x_train[28184]

"It 's a bad sign when you 're rooting for the film to hurry up and get to its subjects ' deaths just so the documentary will be over , but it 's indicative of how uncompelling the movie is unless it happens to cover your particular area of interest ."

In [157]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

23

In [158]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9415387431679699

In [159]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [160]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [161]:
x_train_pad.shape

(61182, 23)

In [162]:
x_test_pad.shape

(15296, 23)

In [163]:
np.array(x_train_tokens[100])

array([   6,   70,    5,  836, 1366, 2223, 3438,   26, 5272, 1192,  170,
          3, 2109, 2110,    2, 1495,    4,  230, 2960])

In [164]:
x_train_pad[100]

array([   0,    0,    0,    0,    6,   70,    5,  836, 1366, 2223, 3438,
         26, 5272, 1192,  170,    3, 2109, 2110,    2, 1495,    4,  230,
       2960])

In [165]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [166]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [167]:
x_train[100]

"'s little to recommend Snow Dogs , unless one considers cliched dialogue and perverse escapism a source of high hilarity"

In [168]:
x_train_pad[100]

array([   0,    0,    0,    0,    6,   70,    5,  836, 1366, 2223, 3438,
         26, 5272, 1192,  170,    3, 2109, 2110,    2, 1495,    4,  230,
       2960])

In [169]:
tokens_to_string(x_train_tokens[100])

"'s little to recommend snow dogs unless one considers cliched dialogue and perverse escapism a source of high hilarity"

In [170]:
model = Sequential()

In [171]:
embedding_size = 50

In [172]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [173]:
model.add(CuDNNGRU(units=16, return_sequences=True))
model.add(CuDNNGRU(units=8, return_sequences=True))
model.add(CuDNNGRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [174]:
optimizer = Adam(lr=1e-3)

In [175]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [176]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 23, 50)            500000    
_________________________________________________________________
cu_dnngru_9 (CuDNNGRU)       (None, 23, 16)            3264      
_________________________________________________________________
cu_dnngru_10 (CuDNNGRU)      (None, 23, 8)             624       
_________________________________________________________________
cu_dnngru_11 (CuDNNGRU)      (None, 4)                 168       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [177]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=100)

Epoch 1/5
61182/61182 [==============================] - 13s 218us/sample - loss: 0.3921 - acc: 0.8185
Epoch 2/5
61182/61182 [==============================] - 12s 200us/sample - loss: 0.2307 - acc: 0.9112
Epoch 3/5
61182/61182 [==============================] - 13s 205us/sample - loss: 0.1815 - acc: 0.9325
Epoch 4/5
61182/61182 [==============================] - 12s 204us/sample - loss: 0.1522 - acc: 0.9437
Epoch 5/5
61182/61182 [==============================] - 12s 201us/sample - loss: 0.1352 - acc: 0.9491


In [178]:
result = model.evaluate(x_test_pad, y_test)

15296/15296 [==============================] - 4s 255us/sample - loss: 0.5154 - acc: 0.8109


In [179]:
result[1]

0.8108656

In [180]:
result[0]

0.5154463212925426

In [181]:
y_pred = model.predict(x=x_test_pad[0:1000])

In [183]:
y_pred[:10]

array([[0.96040404],
       [0.91664636],
       [0.77035856],
       [0.76199806],
       [0.90610325],
       [0.92915344],
       [0.9756737 ],
       [0.95793045],
       [0.8736659 ],
       [0.3727181 ]], dtype=float32)

In [184]:
y_pred = y_pred.T[0]

In [185]:
y_pred[:10]

array([0.96040404, 0.91664636, 0.77035856, 0.76199806, 0.90610325,
       0.92915344, 0.9756737 , 0.95793045, 0.8736659 , 0.3727181 ],
      dtype=float32)

In [186]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [187]:
cls_pred[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [188]:
cls_true = np.array(y_test[0:1000])

In [189]:
incorrect = np.where(cls_pred != cls_true)

In [191]:
incorrect

(array([  9,  13,  27,  28,  29,  30,  31,  37,  38,  39,  57,  58,  62,
         63,  64,  82,  84,  87,  88,  89,  90, 118, 119, 137, 141, 144,
        145, 146, 154, 156, 160, 162, 163, 164, 165, 195, 203, 209, 210,
        215, 224, 226, 230, 232, 239, 242, 243, 244, 248, 249, 250, 254,
        256, 257, 258, 270, 292, 293, 294, 295, 303, 306, 307, 312, 315,
        331, 341, 343, 344, 346, 347, 354, 355, 356, 357, 359, 360, 367,
        375, 393, 394, 405, 406, 408, 409, 437, 438, 439, 440, 441, 443,
        454, 460, 461, 462, 463, 485, 504, 506, 512, 513, 519, 523, 524,
        525, 543, 544, 595, 603, 604, 605, 606, 607, 608, 615, 646, 647,
        648, 649, 650, 651, 654, 655, 663, 678, 685, 686, 687, 688, 694,
        695, 696, 697, 698, 708, 710, 713, 718, 721, 722, 725, 726, 733,
        734, 751, 766, 780, 790, 792, 793, 794, 795, 796, 812, 819, 822,
        823, 829, 830, 831, 834, 835, 847, 856, 857, 870, 871, 874, 875,
        887, 888, 889, 890, 891, 892, 893, 894, 895

In [192]:
incorrect[0]

array([  9,  13,  27,  28,  29,  30,  31,  37,  38,  39,  57,  58,  62,
        63,  64,  82,  84,  87,  88,  89,  90, 118, 119, 137, 141, 144,
       145, 146, 154, 156, 160, 162, 163, 164, 165, 195, 203, 209, 210,
       215, 224, 226, 230, 232, 239, 242, 243, 244, 248, 249, 250, 254,
       256, 257, 258, 270, 292, 293, 294, 295, 303, 306, 307, 312, 315,
       331, 341, 343, 344, 346, 347, 354, 355, 356, 357, 359, 360, 367,
       375, 393, 394, 405, 406, 408, 409, 437, 438, 439, 440, 441, 443,
       454, 460, 461, 462, 463, 485, 504, 506, 512, 513, 519, 523, 524,
       525, 543, 544, 595, 603, 604, 605, 606, 607, 608, 615, 646, 647,
       648, 649, 650, 651, 654, 655, 663, 678, 685, 686, 687, 688, 694,
       695, 696, 697, 698, 708, 710, 713, 718, 721, 722, 725, 726, 733,
       734, 751, 766, 780, 790, 792, 793, 794, 795, 796, 812, 819, 822,
       823, 829, 830, 831, 834, 835, 847, 856, 857, 870, 871, 874, 875,
       887, 888, 889, 890, 891, 892, 893, 894, 895, 897, 898, 90

In [193]:
incorrect = incorrect[0]

In [194]:
len(incorrect)

192

In [195]:
idx = incorrect[0]
idx

9

In [196]:
text = x_test[idx]
text

'should never forget'

In [197]:
y_pred[idx]

0.3727181

In [198]:
cls_true[idx]

1

In [199]:
y_pred[13]

0.9223533

In [200]:
cls_true[13]

0

In [201]:
x_test[13]

'is without doubt an artist of uncompromising vision , but that vision is beginning to feel ,'